In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Keypress

In [49]:
keyPress = pd.read_csv('/Volumes/Untitled/agg_datasets/KeyPressEvent.csv',index_col = 0)
#keyPress = pd.read_csv('./../Our Data/KeyPressEvent.csv',index_col = 0)

/Users/thicorfon/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [50]:
keyPress = keyPress.rename(columns={'PressType':'ActivityID','ActivityID':'PressType'})

In [51]:
keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6).astype(int)
#keyPress['SessionID'] = keyPress.ActivityID

In [5]:
aux = keyPress[['Systime','SessionID']].groupby('SessionID').agg([np.min,np.max]).Systime

In [6]:
aux['diff'] = aux.amax - aux.amin
aux = aux.reset_index()

In [7]:
bla = []
step = 10000
for _,row in aux.iterrows():
    i = 0
    t0 = row.amin
    while (t0+step < row.amax):
        aux_dict = {}
        aux_dict['SessionID'] = row.SessionID
        aux_dict['WindowNumber'] = i
        aux_dict['WindowStart'] = t0
        aux_dict['WindowEnd'] = t0+step
        t0 += step
        i += 1
        bla.append(aux_dict)

In [8]:
fim = pd.DataFrame(bla)

In [9]:
fim.to_csv('skeleton.csv',index=False)

In [2]:
skeleton = pd.read_csv('other_skeleton_10secs.csv')

In [53]:
%%time
aux = keyPress.merge(fim,on='SessionID')

CPU times: user 409 ms, sys: 339 ms, total: 748 ms
Wall time: 845 ms


In [54]:
aux = aux[(aux.Systime >= aux.WindowStart) & (aux.Systime <= aux.WindowEnd)]

In [55]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000


In [57]:
keyPress.to_csv('/Volumes/Untitled/windowed/KeyPressEvent.gz',index=False,compression='gzip')

# Touch Event

In [58]:
keyPress = pd.read_csv('/Volumes/Untitled/agg_datasets/TouchEvent.csv')
#keyPress = pd.read_csv('./../Our Data/KeyboardTouchEvent.csv',index_col = 0)

/Users/thicorfon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6)
#keyPress['SessionID'] = keyPress.ActivityID

In [60]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000


In [61]:
keyPress.to_csv('/Volumes/Untitled/windowed/TouchEvent.csv',index=False,compression='gzip')
#keyPress.to_csv('../Our Data Windowed/TouchEvent.csv',index=False)

# Accelerometer

In [62]:
keyPress = pd.read_csv('/Volumes/Untitled/agg_datasets/Accelerometer.csv',index_col = 0)

/Users/thicorfon/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [63]:
keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6)
#keyPress['SessionID'] = keyPress.ActivityID

In [64]:
%%time
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [65]:
#keyPress.to_csv('../Our Data Windowed/Accelerometer.csv',index=False)
keyPress.to_csv('/Volumes/Untitled/windowed/Accelerometer.gz',index=False,compression='gzip')

# Gyroscope

In [3]:
#keyPress = pd.read_csv('./../Our Data/Gyroscope.csv',index_col = 0)
keyPress = pd.read_csv('/Volumes/Untitled/agg_datasets/Gyroscope.csv',index_col = 0)

/Users/thicorfon/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
keyPress = keyPress.rename(columns={'EventTime':'ActivityID'})

In [5]:
keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6)
#keyPress['SessionID'] = keyPress.ActivityID

In [ ]:
%%time
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000


In [ ]:
#keyPress.to_csv('../Our Data Windowed/Gyroscope.csv',index=False)
keyPress.to_csv('/Volumes/Untitled/windowed/Gyroscope.gz',index=False,compression='gzip')

# Magnetometer

In [ ]:
#keyPress = pd.read_csv('./../Our Data/Magnetometer.csv',index_col = 0)
keyPress = pd.read_csv('/Volumes/Untitled/agg_datasets/Magnetometer.csv',index_col = 0)

In [ ]:
keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6)
#keyPress['SessionID'] = keyPress.ActivityID

In [ ]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

In [ ]:
#keyPress.to_csv('../Our Data Windowed/Magnetometer.csv',index=False)
keyPress.to_csv('/Volumes/Untitled/windowed/Magetometer.gz',index=False,compression='gzip')